In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import pandas as pd

# Choose a fine-tunable model (e.g., distilbert-base-uncased)
model_name = "distilbert-base-uncased"

# Load tokenizer and model for sequence classification
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # 2 for Bechdel test (pass/fail)

# Function to prepare data for training
def prepare_data(data, tokenizer, max_len=512):
  """
  Tokenize text data and prepare labels for training
  """
  encoded_data = tokenizer(data["script"].tolist(), truncation=True, padding="max_length", max_length=max_len)
  labels = data["passed_bechdel"].tolist()
  return encoded_data, labels

# Load datasets
train_data = pd.read_csv("../2_preprocessing/train_lowercase.csv")
val_data = pd.read_csv("../2_preprocessing/validation_lowercase.csv")

# Prepare train and validation data
train_inputs, train_labels = prepare_data(train_data, tokenizer)
val_inputs, val_labels = prepare_data(val_data, tokenizer)

# Training loop (using Adam optimizer and BCEWithLogitsLoss)
from transformers import TrainingArguments, Trainer

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,  # Adjust batch size based on GPU memory
    per_device_eval_batch_size=16,
    num_train_epochs=3,  # Adjust number of epochs
    learning_rate=2e-5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
)

# Define Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_inputs,
    eval_dataset=val_inputs,
    compute_metrics=compute_metrics,  # Define custom metric function (optional)
)

# Train the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./fine-tuned_bechdel_model")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/ucloud/.local/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>={ACCELERATE_MIN_VERSION}'`